In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：自定义训练表格回归模型以进行批量预测并解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练和部署用于批量预测的自定义表格回归模型，并提供解释。

数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。在本教程中使用的数据集版本已经集成到TensorFlow中。训练的模型可以预测房屋的中位价格，单位为1千美元。

### 目标

在本教程中，您将使用Vertex客户端库从Python脚本中创建自定义模型和训练管道，并使用Google预构建的Docker容器。然后，您将对上传的模型进行批量预测并解释结果。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建自定义模型。

执行的步骤包括：

- 创建用于训练模型的Vertex自定义作业。
- 训练TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 上传模型作为Vertex的`Model`资源。
- 进行带解释的批量预测。

### 费用

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用量生成费用估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 *google-cloud-storage* 库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

安装本教程所需的其他软件包。

In [ ]:
! pip3 install -U tabulate $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

### GPU运行时

*确保在有GPU运行时选项的情况下运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的Google Cloud项目

**无论您是在哪种笔记本环境下，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用了结算功能。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex APIs和Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk)已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在此笔记本中所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`为前缀的行作为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改`REGION`变量，该变量用于本笔记本的其余操作。以下是Vertex支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练 Vertex。并非所有地区都支持所有 Vertex 服务。有关每个地区最新支持的信息，请参阅 [Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行直播教程会话，您可能正在使用共享测试帐户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经被验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务账户授权访问项目**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，选择**储存对象管理员**。

点击创建。一个包含您的密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务账户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，都需要执行以下步骤。**

使用 Vertex 客户端库提交自定义训练作业时，您需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 将从该包中运行代码。在本教程中，Vertex 还会在同一个存储桶中保存由您的作业产生的训练模型。然后，您可以基于这个输出创建一个`Endpoint`资源，以便提供在线预测。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中且包括您组织之外的项目中是全局唯一的。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、管道和端点服务端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

设置硬件加速器（例如，GPU），如果有的话，用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器映像和为虚拟机（VM）实例分配的GPU数量。例如，要使用一个具有4个 Nvidia Telsa K80 GPU 分配给每个 VM 的 GPU 容器映像，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`（None，None）`以使用一个在 CPU 上运行的容器映像。

*注意*：在 GPU 支持之前的 TF 2.3 版本之前，将无法加载此教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已修复，这是由在服务功能中生成的静态图操作所致。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）映像

接下来，我们将设置用于训练和预测的Docker容器映像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

请查看[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)获取最新列表。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

请查看[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)获取最新列表。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\]个CPU核心

*备注：以下不支持用于训练：*

 - `standard`：2个vCPU
 - `highcpu`：2、4和8个vCPU

*备注：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始为波士顿房价创建自己的定制模型并进行训练。

## 配置客户端

Vertex客户端库以客户端/服务器模型运作。在您的一侧（Python脚本），您将创建一个客户端，从Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。所以请提前设置好它们。

- 为`Model`资源设置模型服务。
- 为部署设置端点服务。
- 为批处理作业和自定义训练设置作业服务。
- 为提供服务设置预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方式可以使用容器镜像来训练一个自定义模型：

- **使用Google Cloud预构建的容器**。如果使用预构建的容器，您还需要指定一个Python包来安装到容器镜像中。这个Python包包含了训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用您自己的容器，容器需要包含您的训练自定义模型的代码。

准备您的自定义工作规范

现在您的客户准备就绪，您的第一步是为自定义培训工作创建一个工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：您将用于培训的机器类型和数量（单个或分布式）的规范
- `python_package_spec`：将与预构建容器一起安装的Python软件包的规范

###准备您的机器规格

现在为您的自定义训练作业定义机器规格。这告诉 Vertex 为训练提供什么类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型 -- 例如，n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则表示您正在使用 GPU；否则您将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的定制训练作业定义磁盘规格。这告诉 Vertex 在每台机器实例中为训练提供什么类型和大小的磁盘。

  - `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
  - `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您需要为自定义训练作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要准备的这种机器类型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及用于Python模块的命令行参数。

现在，让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的Docker镜像。

- `package_uris`：这是要安装在已准备的实例上的Python训练包的位置（URI）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单独的Python文件，也可以是整个包的zip（归档）文件。在后一种情况下，作业服务将解压缩（解档）内容到Docker镜像中。

- `python_module`：要调用以运行自定义训练作业的Python模块（脚本）。在本例中，您将调用`trainer.task.py` -- 注意，无需添加`.py`后缀。

- `args`：要传递给相应Python模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。告诉训练脚本保存模型工件位置有两种方式：
      - 直接：您将Cloud Storage位置作为命令行参数传递给训练脚本（设置变量 `DIRECT = True`），
      - 间接：服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的纪元数。
  - `"--steps=" + STEPS`：每个纪元的步骤（批次）数量。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单设备或分布式训练的训练分布策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装定制工作规范的完整描述：

- `display_name`：您为此定制工作分配的人类可读名称。
- `job_spec`：定制工作的规范。
    - `worker_pool_specs`：机器虚拟机实例的规范。
    - `base_output_directory`：这告诉服务云存储位置，在变量`DIRECT = False`时保存模型工件的位置。然后，服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训作业组装Python包。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义培训作业的Python脚本。*注意*，当在工作进程池规范中引用它时，我们会将目录斜杠替换为点 (`trainer.task`) 并删除文件后缀 (`.py`)。

#### 包组装

在接下来的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写训练脚本task.py的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量`AIP_MODEL_DIR`获取。
- 从TF.Keras内置数据集中加载波士顿房屋数据集。
- 使用TF.Keras模型API构建简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 训练模型（`fit()`）并指定epochs。
- 将经过训练的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中`f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar包，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用这个辅助函数`create_custom_job`，它接受以下参数：

- `custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，带有以下参数：

- `parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
- `custom_job`：自定义作业的规范。

您将显示`response`对象中返回的一些字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您将保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关定制作业的信息

接下来，使用此辅助函数 `get_custom_job`，它接受以下参数：

- `name`：定制作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，并传入以下参数：

- `name`：定制作业的 Vertex 完全限定标识符。

如果您记得，当您调用 `create_custom_job` 方法时，您在 `response.name` 字段中获得了定制作业的 Vertex 完全限定标识符，并将该标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将结束时间减去开始时间来计算训练模型所需的实际时间。对于您的模型，我们需要知道保存模型的位置，即Python脚本保存在您本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在云存储桶中。现在从云存储桶中加载它，然后您可以执行一些操作，比如评估该模型和进行预测。

要加载模型，您可以使用 TF.Keras 中的 `model.load_model()` 方法，并传递保存模型的云存储路径 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们看看模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`加载波士顿房屋测试（留存）数据，使用`load_data()`方法。这将返回一个元组，包含两个元素。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和相应的标签（业主自住房屋的中位价值）。

您不需要训练数据，这就是为什么我们将其加载为`(_, _)`。

在运行数据进行评估之前，您需要对其进行预处理：

x_test：
1. 将每列中的数据标准化（重新缩放），方法是通过该列的最大值除以每个值。这将用一个介于0和1之间的32位浮点数取代每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 进行模型评估

现在评估定制工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型进行服务

接下来，您将上传您的TF.Keras模型从自定义作业到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型期望的格式。如果您将编码数据发送到Vertex，则您的服务函数会确保数据在传递给模型输入之前在模型服务器上解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将由一个HTTP服务器接收。HTTP服务器从HTTP请求内容中提取预测请求。提取的预测请求将被转发到服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递到服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为基础模型（动态图）的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，标准化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 封装输出给接收应用程序 -- 例如，添加标题，创建JSON对象等。

预处理和后处理函数都会转换为静态图，然后与模型融合。基础模型的输出传递给后处理函数。后处理函数将转换/封装后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建服务函数时，您需要考虑一个问题，即它们作为静态图运行。这意味着，您不能使用需要动态图的TF图操作。如果您这样做，您将在服务函数的编译过程中收到一个错误，提示您正在使用不受支持的EagerTensor。

获取服务功能签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称--这将在您进行预测请求时使用。

您还需要知道服务功能的输入和输出层的名称以构建解释元数据--这将在后续讨论中详细说明。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

### 说明规范

在进行预测时获得解释时，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于对模型进行解释的可解释性算法的规范。您可以选择以下选项之一：
  - Shapley - *注意*，不推荐用于图像数据--可能会运行时间很长
  - XRAI
  - 综合梯度
- `metadata`：这是用于在您的自定义模型上应用算法的规范。

#### 解释参数

让我们首先深入了解可解释性算法的设置。

#### Shapley

将结果的功劳分配给每个特征，并考虑特征的不同排列。该方法提供了对精确 Shapley 值的采样近似。

用例：
  - 对表格数据进行分类和回归。

参数：

- `path_count`：这是算法将处理的特征路径数量。对于精确的 Shapley 值近似需要 M! 条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非常规的特征数量来说，这是计算代价太高。您可以将特征路径的数量减少到 M * `path_count`。

#### 综合梯度

一种基于梯度的方法，用于高效地计算具有与 Shapley 值相同的公理属性的特征归因。

用例：
  - 对表格数据进行分类和回归。
  - 对图像数据进行分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是50步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域，创建一个显着性地图，突出显示图像的相关区域而不是像素。

用例：

  - 对图像数据进行分类。

参数：

- `step_count`：这是近似剩余总和的步数。步数越多，积分近似越准确。一般的经验法则是50步，但随着步数的增加，计算时间也会增加。

在下一个代码单元中，将变量 `XAI` 设置为您将在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们首先深入了解解释元数据，其中包括：

- `outputs`：输出中的一个标量值用于属性 -- 要解释什么。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，我们想要解释0.7。考虑以下公式，其中输出为 `y`，这就是我们想要解释的内容。

    y = f(x)

考虑以下公式，其中输出为 `y` 和 `z`。由于我们只能对一个标量值进行归因，我们必须选择我们想要解释输出 `y` 还是 `z`。在这个例子中假设模型是目标检测，y 和 z 分别是边界框和物体分类。您会想要选择要解释的两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式为：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输出分配的可读性高的名称。一个常见的例子是 "probability"。<br/>
 -  "output_tensor_name"：识别要解释的输出层的键/值字段。 <br/>
 -  [layer]：要解释的输出层。在单任务模型中，如表格回归器，它是模型中最后（最顶部）的一层。
</blockquote>

- `inputs`：用于归因的特征 -- 它们如何影响输出。考虑以下公式，其中 `a` 和 `b` 是特征。我们必须选择讲解如何影响的功能。假设该模型部署用于A/B测试，其中 `a` 是预测的数据项，`b` 标识模型实例是A还是B。您会想要选择 `a`（或一些特征的子集）作为特征，而不选择 `b` 因为它不会影响预测。

    y = f(a, b)

`inputs` 的最小字典格式为：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输入分配的可读性高的名称。一个常见的例子是 "features"。<br/>
 -  "input_tensor_name"：标识特征归因的输入层的键/值字段。 <br/>
 -  [layer]: 特征归因的输入层。在单输入张量模型中，它是模型中最前面（最底部）的一层。
</blockquote>

由于模型的输入为表格数据，您可以指定以下两个附加字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：表示字段值为图像数据。
</blockquote>

由于模型的输入为表格数据，您可以指定以下两个附加字段作为报告/可视化辅助：

<blockquote>
 - "encoding": "BAG_OF_FEATURES"：指示输入为一组表格特征。<br/>
 - "index_feature_mapping": [ feature-names ]：每个特征的人类可读名称列表。在此示例中，我们使用数据集中指定的特征名称。<br/>
 - "modality": "numeric"：表示字段值为数字。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

explanation_spec = aip.ExplanationSpec(metadata=metadata, parameters=parameters)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，将其存储在SavedModel格式中，上传到`Model`服务，该服务将为您的模型实例化一个Vertex`Model`资源实例。一旦完成此操作，您可以像任何其他Vertex`Model`资源实例一样使用`Model`资源实例，例如部署到用于提供预测的`Endpoint`资源。

该辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：我们SavedModel工件的Cloud Storage路径。对于本教程，这是`trainer/task.py`保存模型工件的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了该位置。

辅助函数调用`Model`客户端服务的方法`upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是没有用Vertex`Dataset`资源构建的，您将此保持为空（`''`）。
- `artificat_uri`：模型以SavedModel格式存储的Cloud Storage路径。
- `container_spec`：这是Docker容器的规范，该容器将安装在`Endpoint`资源上，从中`Model`资源将提供预测。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用GPU；否则只分配CPU。
- `explanation_spec`：这是启用模型可解释性的规范。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。您调用`response.result()`，这是一个同步调用，并在Vertex Model资源就绪时返回。

辅助函数返回相应Vertex模型实例上传的完全限定标识符`upload_model_response.model`。您将保存此标识符以供后续步骤中使用，保存在变量`model_to_deploy_id`中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):

    model = aip.Model(
        display_name=display_name,
        artifact_uri=model_uri,
        metadata_schema_uri="",
        explanation_spec=explanation_spec,
        container_spec={"image_uri": image_uri},
    )

    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们仅获取您的模型的模型信息。使用这个辅助函数`get_model`，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用Vertex `Model`客户端服务的方法`get_model`，并传入以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 批量预测的模型部署

现在部署您为批量预测创建的经过训练的 Vertex `Model` 资源。这与部署用于按需预测的 `Model` 资源有所不同。

对于在线预测，您需要：

1. 创建一个 `Endpoint` 资源以部署 `Model` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

3. 向 `Endpoint` 资源发出在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求提供资源。

3. 返回批量预测请求的结果给调用方。

4. 作业服务将为批量预测请求取消提供资源。

## 进行批量预测请求

现在对部署的模型进行批量预测。

In [ ]:
test_item_1 = x_test[0]
test_label_1 = y_test[0]
test_item_2 = x_test[1]
test_label_2 = y_test[1]
print(test_item_1.shape)

制作批量输入文件

现在制作一个批量输入文件，您将把它存储在本地云存储桶中。预测请求中的每个实例都是以下格式的字典条目：

{serving_input: content}

- `serving_input`：底层模型的输入层的名称。
- `content`：测试项的特征值作为列表。

In [ ]:
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {serving_input: test_item_1.tolist()}
    f.write(json.dumps(data) + "\n")
    data = {serving_input: test_item_2.tolist()}
    f.write(json.dumps(data) + "\n")

### 计算实例缩放

您有几种选择来缩放计算实例以处理批量预测请求：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动缩放：批量预测请求分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例将被配置，批量预测请求将均匀分布在它们之间。

- 自动缩放：批量预测请求分配到可伸缩数量的计算实例上。
  - 根据负载情况，将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时要提供和去除的实例数，并将最大（`MAX_NODES`）数量设置为要提供的计算实例数量。

计算实例的最小数量对应于字段`min_replica_count`，最大数量对应于字段`max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

###发出批量预测请求

现在你的两个测试项批次已经准备好了，让我们进行批量请求。使用这个辅助函数 `create_batch_prediction_job`，带有以下参数：

- `display_name`：预测作业的人类可读名称。
- `model_name`：用于 `Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件的云存储路径 -- 这是您在上面创建的。
- `gcs_destination_output_uri_prefix`：服务将写入预测的云存储路径。
- `parameters`：用于提供预测结果的额外过滤参数。

辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，具有以下参数：

- `parent`：Dataset、Model 和 Pipeline 资源的 Vertex 位置根路径。
- `batch_prediction_job`：批处理预测作业的规范。

现在让我们深入了解 `batch_prediction_job` 的规范：

- `display_name`：预测批处理作业的人类可读名称。
- `model`：用于 `Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`：用于批量预测作业的计算资源。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则仅分配 CPU。
  - `starting_replica_count`：最初提供的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数量，您之前设置为变量 `MAX_NODES`。
- `model_parameters`：用于提供预测结果的额外过滤参数。不支持自定义模型的额外参数。
- `input_config`：要预测实例的输入源和格式类型。
 - `instances_format`：批量预测请求文件的格式：`csv` 或 `jsonl`。
 - `gcs_source`：一个或多个云存储路径到您的批量预测请求的列表。
- `output_config`：预测的输出目的地和格式。
 - `prediction_format`：批量预测响应文件的格式：`csv` 或 `jsonl`。
 - `gcs_destination`：预测的输出目的地。

这个调用是异步操作。您将从响应对象打印出一些选择的字段，包括：

- `name`：分配给批量预测作业的 Vertex 完全限定标识符。
- `display_name`：预测批处理作业的人类可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：预测是否提供了 True/False 解释（可解释性）。
- `state`：预测作业的状态（进行中、运行中等）。

由于这个调用将需要一些时间来执行，您很可能会收到 `state` 为 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "boston_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
        "generate_explanation": True,
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数`get_batch_prediction_job`，具有以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定符。

该助手函数调用作业客户服务的`get_batch_prediction_job`方法，具有以下参数：

- `name`：批量预测作业的 Vertex 完全限定符。在本教程中，您将向其传递批量预测作业的 Vertex 完全限定符 -- `batch_job_id`

辅助函数将返回存储预测的 Cloud 存储路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将会是 `JOB_STATE_SUCCEEDED`。

最终，您可以查看存储在您设置为输出的云存储路径上的预测结果。预测结果将以 JSONL 格式存储，您在创建批量预测作业时指定，在以 `prediction` 开头的子文件夹下，该文件夹下将会有一个名为 `prediction.results-xxxxx-of-xxxxx` 的文件。

现在显示（cat）内容。您将会看到多个 JSON 对象，每个对应一个预测结果。

最终，您可以查看存储在您设置为输出的云存储路径上的解释。解释内容将以 JSONL 格式存储，您在创建批量解释作业时指定，在以 `prediction` 开头的子文件夹下，该文件夹下将会有一个名为 `explanation-results-xxxxx-of-xxxxx` 的文件。

让我们显示（cat）内容。每个预测结果都会有一行 - 在本例中，只有一行。该行包含：

- `dense_input`：预测的输入。
- `prediction`：预测的值。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/explanation.results*

        print("Results:")
        ! gsutil cat $folder/explanation.results*

        print("Errors:")
        ! gsutil cat $folder/prediction.errors*
        break
    time.sleep(60)

清理工作

要清理本项目中使用的所有GCP资源，您可以[删除用于教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 终端
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME